## 数据清洗与准备

In [111]:
import pandas as pd
data=pd.read_csv('订单表.csv',encoding='gbk')
data.head()

orders_series = data.set_index('客户ID')['产品名称']
# 按客户ID排序
orders_series.sort_index(inplace=True)

transactions = []
# 首位ID为11000
temp_index = 10999
# 将每个客户ID对应的产品归置一行
for i, v in orders_series.items():
    if i != temp_index:
        temp_set = set()
        temp_index = i
        temp_set.add(v)
        transactions.append(temp_set)
    else:
        temp_set.add(v)

## 1. efficient_aprior

In [120]:
from efficient_apriori import apriori
itemsets, rules = apriori(transactions, min_support=0.03, min_confidence=0.5)
print('频繁项集：', itemsets)
print('\n')
print('关联规则：', rules)

频繁项集： {1: {('骑行服',): 3192, ('山地自行车',): 4089, ('头盔',): 5960, ('车胎和内胎',): 8490, ('旅行自行车',): 2143, ('挡泥板',): 2110, ('帽子',): 2132, ('公路自行车',): 6397, ('水壶和水壶架',): 4548, ('手套',): 1376, ('短裤',): 1019, ('袜子',): 559, ('水袋背包',): 719, ('清洁剂',): 875, ('背心',): 557}, 2: {('头盔', '山地自行车'): 1388, ('头盔', '挡泥板'): 742, ('头盔', '旅行自行车'): 1058, ('头盔', '车胎和内胎'): 2921, ('头盔', '骑行服'): 923, ('山地自行车', '挡泥板'): 898, ('山地自行车', '旅行自行车'): 1246, ('山地自行车', '车胎和内胎'): 1107, ('山地自行车', '骑行服'): 679, ('挡泥板', '车胎和内胎'): 723, ('车胎和内胎', '骑行服'): 1150, ('公路自行车', '头盔'): 2051, ('公路自行车', '山地自行车'): 2053, ('公路自行车', '帽子'): 673, ('公路自行车', '水壶和水壶架'): 1590, ('公路自行车', '骑行服'): 1154, ('头盔', '帽子'): 678, ('头盔', '水壶和水壶架'): 1453, ('山地自行车', '水壶和水壶架'): 1246, ('帽子', '水壶和水壶架'): 746, ('帽子', '骑行服'): 689, ('水壶和水壶架', '骑行服'): 774, ('帽子', '车胎和内胎'): 760, ('旅行自行车', '水壶和水壶架'): 780, ('水壶和水壶架', '车胎和内胎'): 812, ('头盔', '手套'): 687, ('手套', '车胎和内胎'): 677, ('公路自行车', '车胎和内胎'): 1450}, 3: {('头盔', '山地自行车', '旅行自行车'): 677, ('公路自行车', '头盔', '山地自行车'): 726, ('公路自行车', '山地自行车', '水

## 2. mlxtend

In [47]:
#  传入模型的数据需要满足特定的格式，转换为bool值
from mlxtend.preprocessing import TransactionEncoder#
te = TransactionEncoder()
df_tf = te.fit_transform(transactions)
# 将 True、False 转换为 0、1 
df_01 = df_tf.astype('int')
encoded_df= pd.DataFrame(df_01,columns=te.columns_)

In [94]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = apriori(encoded_df, min_support=0.05, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support',ascending=False)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules= rules.sort_values(by='lift',ascending=False)
print("频繁项集：", frequent_itemsets)
print("关联规则：", rules)

频繁项集：      support         itemsets
9   0.459316          (车胎和内胎)
0   0.346083          (公路自行车)
1   0.322441             (头盔)
7   0.246051         (水壶和水壶架)
2   0.221218          (山地自行车)
10  0.172690            (骑行服)
19  0.158029      (头盔, 车胎和内胎)
6   0.115938          (旅行自行车)
3   0.115343             (帽子)
5   0.114153            (挡泥板)
12  0.111069   (山地自行车, 公路自行车)
11  0.110961      (头盔, 公路自行车)
13  0.086020  (水壶和水壶架, 公路自行车)
18  0.078609     (头盔, 水壶和水壶架)
14  0.078446   (车胎和内胎, 公路自行车)
16  0.075092      (山地自行车, 头盔)
4   0.074443             (手套)
20  0.067410   (山地自行车, 旅行自行车)
21  0.067410  (山地自行车, 水壶和水壶架)
15  0.062432     (骑行服, 公路自行车)
23  0.062216     (骑行服, 车胎和内胎)
22  0.059890   (山地自行车, 车胎和内胎)
17  0.057239      (头盔, 旅行自行车)
8   0.055129             (短裤)
关联规则：    antecedents consequents  antecedent support  consequent support   support  \
9      (旅行自行车)     (山地自行车)            0.115938            0.221218  0.067410   
8      (山地自行车)     (旅行自行车)            0.221218            0.115938  0.067410  